In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install captum
!pip install fvcore

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import visualization as viz
from fvcore.nn import FlopCountAnalysis, flop_count_table
from captum.attr import IntegratedGradients, LayerConductance, DeepLift, LayerDeepLift,LayerIntegratedGradients
import itertools
import pandas as pd
import os
import random
import numpy as np
from itertools import product
from torch.utils.data import SequentialSampler, RandomSampler
from torch.utils.data import DataLoader

In [4]:
method_names = ["LayerIntegratedGradients", "LayerDeepLift"]
INPUT_SHAPE= (1, 1, 28, 28)
Training_Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Feature_Attribution_Device = torch.device("cpu")
NUM_EPOCHS=1
NUM_CLASSES=10
IN_CHANNELS=1

# Model structure defination

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Inception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Inception, self).__init__()

        self.conv1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.conv1x1_3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=1),
            nn.Conv2d(8, 16, kernel_size=3, padding=1)
        )
        self.conv1x1_5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=1),
            nn.Conv2d(8, 16, kernel_size=5, padding=2)
        )
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

        # The following size needs to be calculated based on the output dimensions of the concatenated tensors
        # For example, for 28x28 input images (as in MNIST), the size might be different
        self.fc = nn.Sequential(
            nn.Linear(38416, 256),  # Replace XXXX with the correct input size
            nn.ReLU(),
            nn.Linear(256, num_classes)  # Number of output classes
        )

    def forward(self, x):
        x.requires_grad_(True)
        x1 = self.conv1x1(x)
        x2 = self.conv1x1_3x3(x)
        x3 = self.conv1x1_5x5(x)
        x4 = self.pool(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)  # Concatenating along the channel dimension
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x

# Get dataset

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.FashionMNIST('.', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('.', train=False, download=True, transform=transform)

# FLOP Count

In [7]:
def count_flops(model, input_size):
    inputs = torch.randn(input_size)
    flops = FlopCountAnalysis(model, inputs)
    return flop_count_table(flops)

inception_model = Inception(1, 10)
# Assuming the input size for CIFAR-10 (batch size, channels, height, width)
input_size = (1, 1, 28, 28)
flops_table = count_flops(inception_model, input_size)
print(flops_table)

| module                  | #parameters or shape   | #flops   |
|:------------------------|:-----------------------|:---------|
| model                   | 9.842M                 | 13.274M  |
|  conv1x1                |  32                    |  12.544K |
|   conv1x1.weight        |   (16, 1, 1, 1)        |          |
|   conv1x1.bias          |   (16,)                |          |
|  conv1x1_3x3            |  1.184K                |  0.909M  |
|   conv1x1_3x3.0         |   16                   |   6.272K |
|    conv1x1_3x3.0.weight |    (8, 1, 1, 1)        |          |
|    conv1x1_3x3.0.bias   |    (8,)                |          |
|   conv1x1_3x3.1         |   1.168K               |   0.903M |
|    conv1x1_3x3.1.weight |    (16, 8, 3, 3)       |          |
|    conv1x1_3x3.1.bias   |    (16,)               |          |
|  conv1x1_5x5            |  3.232K                |  2.515M  |
|   conv1x1_5x5.0         |   16                   |   6.272K |
|    conv1x1_5x5.0.weight |    (8, 1, 1,

# Train and attribution functions

train and eval function

In [8]:
def train(epoch, model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")

    return train_accuracy

def test(epoch, model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

    return test_accuracy


functions for calculate attribution

In [9]:
def print_ig(test_loader, model, device):
    # Move the model to the device (CPU or CUDA)
    model.to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Get a single batch from the test loader
    inputs, target_class = next(iter(test_loader))
    inputs = inputs.to(device)

    ig_attributions = {}

    # Iterate through each named module and compute attributions for Conv2d layers with learnable parameters
    for layer_name, layer_module in model.named_modules():
        # Check if the layer is a Conv2d layer with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerIntegratedGradients for the layer
            lig = LayerIntegratedGradients(model, layer_module)

            # Compute the attributions for the current layer
            try:
                attributions = lig.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue

            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(f'Attribution: {attributions.cpu().detach().numpy().sum()}')

            # Store the sum of attributions in the dictionary
            ig_attributions[layer_name] = attributions.cpu().detach().numpy().sum()

            # Free up memory
            del attributions, lig

    return ig_attributions

# Usage example:
# ig_attributions = print_ig(test_loader, model, device)


In [10]:
def print_deeplift(test_loader, model, device):
    # Move the model to the specified device and set it to evaluation mode
    model.to(device).eval()

    # Get a batch of data from the loader
    inputs, target_class = next(iter(test_loader))
    inputs, target_class = inputs.to(device), target_class.to(device)

    dl_attributions = {}

    # Now compute the attributions for Conv2d layers
    for layer_name, layer_module in model.named_modules():
        # Skip the whole model's container and focus on Conv2d layers with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerDeepLift with the current layer
            ldl = LayerDeepLift(model, layer_module)

            # Compute the attributions for the current layer
            '''try:
                attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue'''
            attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))


            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(attributions_ldl.cpu().data.numpy().sum())

            dl_attributions[layer_name] = attributions_ldl.cpu().data.numpy().sum()

            del attributions_ldl, ldl

    return dl_attributions

# Usage example:
# dl_attributions = print_deeplift(test_loader, model, device)


# Possible Hyperparameter grid search creation

in hyperparams_list_dict, each hyperparameter has corresponding possible choices as a list, during experiment, given hyperparams sequence, location each hyperperameter's location using hyperparameter encoding function to convert strings or classes back into their index in hyperparams_list_dict

In [11]:
'''sample_hyperparams_list_dict = {
    'initial_lr': [],
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64
}'''

"sample_hyperparams_list_dict = {\n    'initial_lr': [],\n    'optimizer': torch.optim.Adam,  # Example optimizer\n    'criterion': torch.nn.CrossEntropyLoss(),\n    'train_data_used': 0.8,\n    'train_set_shuffle': True,\n    'train_batch_size': 64\n}"

In [12]:
def generate_hyperparameter_combinations(hyperparams):
    """
    Generate a sequence of hyperparameter combinations.

    :param hyperparams: A dictionary where keys are the names of hyperparameters,
                        and values are lists of possible choices for each hyperparameter.
    :return: A list of dictionaries, each representing a unique combination of hyperparameters.
    """
    # Extract the hyperparameter names and their corresponding choices
    keys, values = zip(*hyperparams.items())

    # Generate all possible combinations of hyperparameter values
    all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    return all_combinations

# Automated Experiments

In [13]:
def get_data_loader(hyperparams, train_dataset, test_dataset):
  shuffle = hyperparams['train_set_shuffle']
  train_batch_size = hyperparams["train_batch_size"]
  train_data_used_num = int(len(train_dataset) * hyperparams["train_data_used"])
  train_indices = torch.randperm(len(train_dataset))[:train_data_used_num]

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  return train_loader, test_loader

In [14]:
def run_experiments(hyperparams, train_loader, test_loader, num_epochs=NUM_EPOCHS, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS,):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc




    dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
    print("\n")

    ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
    print("\n")
    return dl_attributions, ig_attributions, train_accuracy, test_accuracy

# Example Usage
hyperparams = {
    'initial_lr': 0.001,
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64

}

#train_loader, test_loader = get_data_loader(hyperparams, train_dataset, test_dataset)
#print(run_experiments(num_epochs=1, hyperparams=hyperparams, train_loader=train_loader, test_loader=test_loader))

# Functions for saving attribution

In [15]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param attribution_function: Function to compute feature layer attribution.
    :param csv_file: Path to the CSV file for saving results.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(10):  # For each run index
            for method in ['deeplift', 'integrated_gradients']:  # For each method
                print("runtime"+str(i))
                # Prepare data for checking if it's already processed
                combo_check = combo.copy()
                combo_check['method'] = method
                combo_check['run'] = i

                # Check if this specific combination is already processed
                if not existing_data.empty and (existing_data[list(combo_check.keys())] == list(combo_check.values())).all(axis=1).any():
                    continue  # Skip if combination is already processed

                # Set seed for reproducibility
                random.seed(i)
                np.random.seed(i)
                torch.manual_seed(i)
                if torch.cuda.is_available():
                    torch.cuda.manual_seed_all(i)

                train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Initialize model
                # Compute attributions
                attr = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader)

                # Prepare data for saving
                combo_results = combo.copy()

                # loop over the recorded result from one run, and add these into our row data
                for d in attr:
                  combo_results.update(d)

                combo_results['method'] = method
                combo_results['run'] = i

                # Append results to the existing data
                existing_data = existing_data.append(combo_results, ignore_index=True)

                # Save the data to CSV after each run
                existing_data.to_csv(csv_file, index=False)



# Experiments

In [16]:
hyperparams_choice_list = {
    'initial_lr': [0.001, 0.01],
    'optimizer': [torch.optim.Adam], # Example optimizer
    'criterion': [torch.nn.CrossEntropyLoss()],
    'train_data_used': [1, 0.5],
    'train_set_shuffle': [True,False],
    'train_batch_size': [64, 32, 16]
}

combinations = generate_hyperparameter_combinations(hyperparams_choice_list)
for combo in combinations:
    print(combo)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': False, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': False, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'trai

In [17]:
file_path=os.path.join('/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage2/fashion', 'fashion_result.csv')
print(file_path)

/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage2/fashion/fashion_result.csv


In [18]:
run_experiments_and_save(combinations, train_dataset, test_dataset, csv_file=file_path)




Combination:
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
runtime0
Epoch 0: Train Loss = 0.5111, Train Accuracy = 82.45%
Epoch 0: Test Loss = 0.4271, Test Accuracy = 84.27%
Layer: conv1x1
268.79617
Layer: conv1x1_3x3.0
105.783264
Layer: conv1x1_3x3.1
105.78324
Layer: conv1x1_5x5.0
124.801186
Layer: conv1x1_5x5.1
124.80117


Layer: conv1x1
Attribution: 335.56694109731296
Layer: conv1x1_3x3.0
Attribution: 130.3482681608265
Layer: conv1x1_3x3.1
Attribution: 130.34827129707259
Layer: conv1x1_5x5.0
Attribution: 155.53220190152228
Layer: conv1x1_5x5.1
Attribution: 155.5322001106402


runtime0


<ipython-input-15-0701789aeb08>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.5111, Train Accuracy = 82.45%
Epoch 0: Test Loss = 0.4271, Test Accuracy = 84.27%
Layer: conv1x1
268.7962
Layer: conv1x1_3x3.0
105.783264
Layer: conv1x1_3x3.1
105.78324
Layer: conv1x1_5x5.0
124.80117
Layer: conv1x1_5x5.1
124.80116


Layer: conv1x1
Attribution: 335.5669466754835
Layer: conv1x1_3x3.0
Attribution: 130.3482622755584
Layer: conv1x1_3x3.1
Attribution: 130.34826619385157
Layer: conv1x1_5x5.0
Attribution: 155.53219441522955
Layer: conv1x1_5x5.1
Attribution: 155.5321928447569


runtime1


<ipython-input-15-0701789aeb08>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.5334, Train Accuracy = 82.04%
Epoch 0: Test Loss = 0.4499, Test Accuracy = 83.46%
Layer: conv1x1
197.1001
Layer: conv1x1_3x3.0
105.72963
Layer: conv1x1_3x3.1
105.72962
Layer: conv1x1_5x5.0
188.96466
Layer: conv1x1_5x5.1
188.96474


Layer: conv1x1
Attribution: 258.5850596954518
Layer: conv1x1_3x3.0
Attribution: 135.12730826429345
Layer: conv1x1_3x3.1
Attribution: 135.127308285243
Layer: conv1x1_5x5.0
Attribution: 231.52024005855438
Layer: conv1x1_5x5.1
Attribution: 231.52024780350493


runtime1


...

<ipython-input-15-0701789aeb08>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 2.4817, Train Accuracy = 33.14%
Epoch 0: Test Loss = 2.3047, Test Accuracy = 10.00%
Layer: conv1x1
0.0
Layer: conv1x1_3x3.0
0.0
Layer: conv1x1_3x3.1
0.0
Layer: conv1x1_5x5.0
0.0
Layer: conv1x1_5x5.1
0.0


Layer: conv1x1
Attribution: 0.0
Layer: conv1x1_3x3.0
Attribution: 0.0
Layer: conv1x1_3x3.1
Attribution: 0.0
Layer: conv1x1_5x5.0
Attribution: 0.0
Layer: conv1x1_5x5.1
Attribution: 0.0


runtime9


<ipython-input-15-0701789aeb08>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 1.9093, Train Accuracy = 56.51%
Epoch 0: Test Loss = 0.8684, Test Accuracy = 69.88%
Layer: conv1x1
479.7795
Layer: conv1x1_3x3.0
233.7886
Layer: conv1x1_3x3.1
233.78854
Layer: conv1x1_5x5.0
95.14405
Layer: conv1x1_5x5.1
95.144005


Layer: conv1x1
Attribution: 624.4039245967563
Layer: conv1x1_3x3.0
Attribution: 276.65951013414775
Layer: conv1x1_3x3.1
Attribution: 276.65952175801505
Layer: conv1x1_5x5.0
Attribution: 107.09925851681022
Layer: conv1x1_5x5.1
Attribution: 107.09926203272487




<ipython-input-15-0701789aeb08>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
